#Configuration Parameters

In [0]:
training_set_path='/content/drive/My Drive/Metadata/one_percent_train.txt'
val_set_path='/content/drive/My Drive/Metadata/one_percent_val.txt'
test_set_path='/content/drive/My Drive/Metadata/one_percent_test.txt'
model_path='/content/drive/My Drive/models/one_percent/'

n_splits = 1
set_info={"allBands":[0,1,2,3,4,5,6,7,8,9]} # number of bands

num_classes=2
image_shape=(512,512,10)
padding=((0,0),(0,0))
batch_size=5
epochs=50
learning_rate= 0.02

training_set_size=770
val_set_size=100
test_set_size=100
# optimizer='sgd'
loss_function='categorical_crossentropy'
metrics=['accuracy']

#Creating encoding, decoding and max pooling layers


In [0]:
from keras.layers import Activation,BatchNormalization,Conv2D
from keras.engine.topology import Layer
import keras.backend.tensorflow_backend as K
from keras.optimizers import Adam, SGD

class MaxPoolingWithIndices(Layer):
    def __init__(self, pool_size,strides,padding='SAME',**kwargs):
        super(MaxPoolingWithIndices, self).__init__(**kwargs)
        self.pool_size=pool_size
        self.strides=strides
        self.padding=padding
        return
    def call(self,x):
        pool_size=self.pool_size
        strides=self.strides
        if isinstance(pool_size,int):
            ps=[1,pool_size,pool_size,1]
        else:
            ps=[1,pool_size[0],pool_size[1],1]
        if isinstance(strides,int):
            st=[1,strides,strides,1]
        else:
            st=[1,strides[0],strides[1],1]
        output1,output2=K.tf.nn.max_pool_with_argmax(x,ps,st,self.padding)
        return [output1,output2]
    def compute_output_shape(self, input_shape):
        if isinstance(self.pool_size,int):
            output_shape=(input_shape[0],input_shape[1]//self.pool_size,input_shape[2]//self.pool_size,input_shape[3])
        else:
            output_shape=(input_shape[0],input_shape[1]//self.pool_size[0],input_shape[2]//self.pool_size[1],input_shape[3])
        return [output_shape,output_shape]


class UpSamplingWithIndices(Layer):
    def __init__(self, **kwargs):
        super(UpSamplingWithIndices, self).__init__(**kwargs)
        return
    def call(self,x):
        argmax=K.cast(K.flatten(x[1]),'int32')
        max_value=K.flatten(x[0])
        with K.tf.variable_scope(self.name):
            input_shape=K.shape(x[0])
            batch_size=input_shape[0]
            image_size=input_shape[1]*input_shape[2]*input_shape[3]
            output_shape=[input_shape[0],input_shape[1]*2,input_shape[2]*2,input_shape[3]]
            indices_0=K.flatten(K.tf.matmul(K.reshape(K.tf.range(batch_size),(batch_size,1)),K.ones((1,image_size),dtype='int32')))
            indices_1=argmax%(image_size*4)//(output_shape[2]*output_shape[3])
            indices_2=argmax%(output_shape[2]*output_shape[3])//output_shape[3]
            indices_3=argmax%output_shape[3]
            indices=K.tf.stack([indices_0,indices_1,indices_2,indices_3])
            output=K.tf.scatter_nd(K.transpose(indices),max_value,output_shape)
            return output
    def compute_output_shape(self, input_shape):
        return input_shape[0][0],input_shape[0][1]*2,input_shape[0][2]*2,input_shape[0][3]

def CompositeConv(inputs,num_layers,num_features):
    output=inputs
    if isinstance(num_features,int):
        for i in range(num_layers):
            output=Conv2D(num_features,(7,7),padding='same')(output)
            output=BatchNormalization(axis=3)(output)
            output=Activation('relu')(output)
        return output
    for i in range(num_layers):
        output=Conv2D(num_features[i],(7,7),padding='same')(output)
        output=BatchNormalization(axis=3)(output)
        output=Activation('relu')(output)
    return output


#Creating SegNet model


In [0]:
from keras.models import Model
from keras.layers import Activation,Input,ZeroPadding2D,Cropping2D


def create_model():
    inputs=Input(shape=image_shape)

    x = ZeroPadding2D(padding)(inputs)

    x=CompositeConv(x,2,64)
    x,argmax1=MaxPoolingWithIndices(pool_size=2,strides=2)(x)
    
    x=CompositeConv(x,2,64)
    x,argmax2=MaxPoolingWithIndices(pool_size=2,strides=2)(x)
    
    x=CompositeConv(x,3,64)
    x,argmax3=MaxPoolingWithIndices(pool_size=2,strides=2)(x)

    x=CompositeConv(x,3,64)
    x,argmax4=MaxPoolingWithIndices(pool_size=2,strides=2)(x)

    x=CompositeConv(x,3,64)
    x,argmax5=MaxPoolingWithIndices(pool_size=2,strides=2)(x)

    x=UpSamplingWithIndices()([x,argmax5])
    x=CompositeConv(x,3,64)

    x=UpSamplingWithIndices()([x,argmax4])
    x=CompositeConv(x,3,64)

    x=UpSamplingWithIndices()([x,argmax3])
    x=CompositeConv(x,3,64)

    x=UpSamplingWithIndices()([x,argmax2])
    x=CompositeConv(x,2,64)
    
    x=UpSamplingWithIndices()([x,argmax1])
    x=CompositeConv(x,2,[64,num_classes])

    x=Activation('softmax')(x)

    y=Cropping2D(padding)(x)
    my_model=Model(inputs=inputs,outputs=y)
    
    return my_model

#Read training image chunks in batches 


In [0]:
import imageio
import numpy as np
from keras.utils import to_categorical


def read_image_batch(image_list, batch_size, channel_list):
    while True:
        l=len(image_list)
        num_batch=l//batch_size
        if num_batch*batch_size<l:
            num_batch+=1
        for i in range(num_batch):
            batch_set=image_list[batch_size*i:min(batch_size*(i+1),l),:]
            batch_set=[batch_set[bs] for bs in range(len(batch_set))]
            X=np.array([np.load(line[0][0:]) for line in batch_set])
            labels=np.array([np.load(line[1][0:]) for line in batch_set])
            y=to_categorical(labels,num_classes)
            X = X[:, :, :, channel_list]
            yield tuple((X, y))


#Training model 


In [115]:
import argparse
from keras.utils import multi_gpu_model
from keras.models import Model
from keras.layers import Activation,Input
import os

def getSubsets(data_path, set_info, subset_size):
  image_list=[]
  with open(data_path,"r") as f:
    for line in f.readlines():
      arr=[]
      str_array=line.split(" ")
      arr.append(str_array[0]+" "+str_array[1])
      arr.append(str_array[2]+" "+str_array[3][:-1])
      image_list.append(arr)
     
  image_list = np.asarray(image_list)
  np.random.seed(1)
  np.random.shuffle(image_list)
  data = []
  print(image_list.shape)
  jump = int(subset_size - ((subset_size*n_splits)-image_list.shape[0])/2)
  offset = image_list.shape[0]
  if n_splits>1:
      offset = image_list.shape[0]-subset_size
  for i in range(0, offset, jump):
      subset = image_list[i: i+subset_size,]
      print(subset.shape)
      for key, value in set_info.items():        
          data.append(read_image_batch(subset, batch_size, value))
  return np.asarray(data)
            

def main(args):
    train_data = getSubsets(training_set_path, set_info, training_set_size)
    val_data = getSubsets(val_set_path, set_info, val_set_size)
  
    key=0
    for i in range(0, len(train_data)):
      my_model=create_model()
      my_model.compile(optimizer=args['opt'],loss=loss_function,metrics=metrics)
      if args['resume']:
            latest_model = args["load"]+"_best_lr_"+str(learning_rate)
            print("Last Model picked up: " + latest_model) 
            my_model.load_weights(model_path+latest_model)
      min_loss = 1.0
      for j in range(11,epochs):
        hist = my_model.fit_generator(train_data[i],
                                steps_per_epoch=(training_set_size+1)//batch_size,
                                epochs=1,validation_data=val_data[i],
                                validation_steps=(val_set_size+1)//batch_size)
        val_loss = hist.history["val_loss"]
        if j%5 == 0:
          my_model.save_weights(model_path+args['save']+"_epoch"+str(j)+"_lr_"+str(learning_rate));  
        if val_loss[0] < min_loss:
          min_loss = val_loss[0]
          my_model.save_weights(model_path+args['save']+"_best_lr_"+str(learning_rate))
        print("model fit epoch: "+str(j))
        

args = {"save": "model_allBands", "resume":False, "load": "model_allBands", "opt": SGD(lr=learning_rate)}
main(args)


(770, 2)
(770, 2)
(100, 2)
(100, 2)
Epoch 1/1
154/154 [==============================] - 238s 2s/step - loss: 0.4742 - acc: 0.8911 - val_loss: 0.3676 - val_acc: 0.9193
model fit epoch: 11
Epoch 1/1
154/154 [==============================] - 200s 1s/step - loss: 0.3281 - acc: 0.9284 - val_loss: 0.3085 - val_acc: 0.9198
model fit epoch: 12
Epoch 1/1
154/154 [==============================] - 199s 1s/step - loss: 0.2707 - acc: 0.9290 - val_loss: 0.2877 - val_acc: 0.9199
model fit epoch: 13
Epoch 1/1
154/154 [==============================] - 200s 1s/step - loss: 0.2417 - acc: 0.9291 - val_loss: 0.2752 - val_acc: 0.9198
model fit epoch: 14
Epoch 1/1
154/154 [==============================] - 199s 1s/step - loss: 0.2197 - acc: 0.9293 - val_loss: 0.2047 - val_acc: 0.9200
model fit epoch: 15
Epoch 1/1
154/154 [==============================] - 199s 1s/step - loss: 0.2009 - acc: 0.9313 - val_loss: 0.2462 - val_acc: 0.9200
model fit epoch: 16
Epoch 1/1
154/154 [==============================] -

#define a list of test image chunks 


In [0]:

test_names=[]

with open(test_set_path,"r") as f:
  test_image_list=[]
  for line in f.readlines():
    arr=[]
    str_array=line.split(" ")
    arr.append(str_array[0]+" "+str_array[1])
    arr.append(str_array[2]+" "+str_array[3][:-1])
    test_image_list.append(arr)
    test_names.append(line)

test_image_list=np.asarray(test_image_list)

#read test chunks in batches using the list defined above 


In [0]:

import imageio
import numpy as np
from keras.utils import to_categorical

def read_test_image_batch(image_list, batch_size, channel_list):
    while True:
        l=len(image_list)
        num_batch=l//batch_size
        if num_batch*batch_size<l:
            num_batch+=1
        for i in range(num_batch):
            batch_set=image_list[batch_size*i:min(batch_size*(i+1),l),:]
            batch_set=[batch_set[bs] for bs in range(len(batch_set))]
            X=np.array([np.load(line[0][0:]) for line in batch_set])
            labels=np.array([np.load(line[1][0:]) for line in batch_set])
            y=to_categorical(labels,num_classes)
            X = X[:, :, :, channel_list]
            yield tuple((X, y))




#testing model


In [0]:
import argparse
from keras.models import Model
from keras.layers import Activation,Input
from scipy.stats import mode
import os


class_labels=[]


for i in os.listdir(model_path):
  if(i==".ipynb_checkpoints"):
    continue

  key=0
  #if(key!=0):
  #   continue

  print(model_path+i)
  my_model=create_model()
  my_model.compile(optimizer,loss=loss_function,metrics=metrics)
  my_model.load_weights(model_path+i)
  
  test_data=read_test_image_batch(test_image_list,batch_size,set_info[key])
  probs=my_model.predict(test_data,steps=(test_set_size+1)//batch_size)
  print("\nprobs")
  print(probs.shape)
  class_labels.append(probs.argmax(axis=-1))

hist = np.histogram(class_labels[0])
class_labels=np.asarray(class_labels)
u,indices=np.unique(class_labels,return_inverse=True)
final_labels=u[np.argmax(np.apply_along_axis(np.bincount,0,indices.reshape(class_labels.shape),None,np.max(indices)+1),axis=0)]

for i in range(test_set_size):
  line=test_names[i]
  print(line)
  str_array=line.split(" ")
  scene_id=str_array[1][18:40]
  out_str=str_array[1][40:-4]+"_output.npy"
  print(out_str)
  np.save('/content/drive/My Drive/new_test_outputs_key_0/'+scene_id+out_str,final_labels[i])
  

In [0]:
"""
This script takes a list of scene ids and creates a file that can be used as input for a segnet model
@param string chunk_dir: The abspath base directory where each set of chunks for a scene has its own dir named with its sceneID
@param list scene_ids: A list of sceneIDs that exist in the chunk_dir. The chunks of these scenes will be used in the file.
@param string out_path: The abspath where the resulting file should be saved.
@return int lines_written: the total number of lines (corresponding to a data and label chunk path) in the file.
file format:
/path/to/scene_chunk.npy /path/to/scene_chunk_label.npy
/path/to/scene_chunk.npy /path/to/scene_chunk_label.npy
/path/to/scene_chunk.npy /path/to/scene_chunk_label.npy
...
"""

import os

def make_segnet_input_file(chunk_dir, scene_ids, out_path):
    existing_scenes = [i for i in os.listdir(chunk_dir) if os.path.isdir(os.path.join(chunk_dir, i))]
    # filter out ids that don't exist in the given dir
    scene_ids = [i for i in scene_ids if i in existing_scenes]
    print(scene_ids)

    lines_to_write = []

    for i in scene_ids:
        scene_dir = os.path.join(chunk_dir, i)
        for j in os.listdir(scene_dir):
            if j[-9:] != "label.npy":
                data_path = os.path.join(scene_dir, j)
                file_split = os.path.splitext(j)
                label_path = os.path.join(scene_dir, file_split[0] + "_label" + file_split[1])

                lines_to_write.append("{} {}\n".format(data_path, label_path))

    with open(out_path, 'w+') as output:
        output.writelines(lines_to_write)

    return len(lines_to_write)

#Mount Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
